# RouterR1 - Inference

This notebook demonstrates how to use **RouterR1** for agentic routing and reasoning.

## Overview

RouterR1 is an agentic router that performs R1-like reasoning with iterative search and routing.
It uses vLLM for local inference and an external routing API for model selection.

**Key Features**:
- Agentic reasoning with iterative search
- Pre-trained model (no training required)
- Uses vLLM for efficient GPU inference
- Integrates with external routing pool API

**Requirements**:
- CUDA-enabled GPU
- vLLM library
- OpenAI-compatible API endpoint

## 1. Environment Setup

In [ ]:
# For Google Colab: Ensure GPU runtime is enabled
# Runtime -> Change runtime type -> Hardware accelerator -> GPU (T4 or better)

import os

if 'COLAB_GPU' in os.environ:
    !git clone https://github.com/ulab-uiuc/LLMRouter.git
    %cd LLMRouter
    !pip install -e .
    !pip install vllm transformers pyyaml openai

In [ ]:
import os
import sys
from pathlib import Path

PROJECT_ROOT = Path(os.getcwd()).parent.parent
if str(PROJECT_ROOT) not in sys.path:
    sys.path.insert(0, str(PROJECT_ROOT))

os.chdir(PROJECT_ROOT)
print(f"Working directory: {os.getcwd()}")

In [ ]:
import torch

# Check CUDA availability (required for RouterR1)
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"GPU count: {torch.cuda.device_count()}")
else:
    print("WARNING: RouterR1 requires CUDA. Please enable GPU runtime.")

In [ ]:
from llmrouter.utils import setup_environment
import yaml

setup_environment()
print("Environment setup complete!")

## 2. Configuration

RouterR1 requires the following configuration parameters:

| Parameter | Description | Required |
|-----------|-------------|----------|
| `model_id` | HuggingFace model ID for reasoning | Yes |
| `api_base` | Routing API endpoint URL | Yes |
| `api_key` | API key for routing service | Yes |

**Note**: Create a config file or set these parameters programmatically.

In [ ]:
# Create configuration for RouterR1
# Replace with your actual API credentials

router_r1_config = {
    "data_path": {
        "query_data_train": "data/example_data/query_data/default_query_train.jsonl",
        "query_data_test": "data/example_data/query_data/default_query_test.jsonl",
        "routing_data_train": "data/example_data/routing_data/default_routing_train_data.jsonl",
        "routing_data_test": "data/example_data/routing_data/default_routing_test_data.jsonl",
        "llm_data": "data/example_data/llm_candidates/default_llm.json"
    },
    "hparam": {
        "model_id": "Qwen/Qwen2.5-3B-Instruct",  # Pre-trained model for reasoning
        "api_base": "https://api.openai.com/v1",  # Replace with your API endpoint
        "api_key": os.environ.get("OPENAI_API_KEY", "your-api-key-here")  # Replace with your API key
    }
}

# Save config to temporary file
CONFIG_PATH = "configs/model_config_train/router_r1_temp.yaml"
os.makedirs(os.path.dirname(CONFIG_PATH), exist_ok=True)

with open(CONFIG_PATH, 'w') as f:
    yaml.dump(router_r1_config, f, default_flow_style=False)

print("Configuration:")
print("=" * 50)
print(yaml.dump(router_r1_config, default_flow_style=False))

## 3. Initialize Router

In [ ]:
from llmrouter.models.router_r1 import RouterR1

# Initialize RouterR1 (requires valid API credentials)
try:
    router = RouterR1(yaml_path=CONFIG_PATH)
    print("RouterR1 initialized successfully!")
    print(f"Model ID: {router.model_id}")
except Exception as e:
    print(f"Error initializing RouterR1: {e}")
    print("Please ensure you have valid API credentials configured.")

## 4. Single Query Routing

RouterR1 performs agentic reasoning with iterative search and routing.
The process includes:
1. Initial prompt generation
2. Iterative reasoning with search queries
3. External routing API calls
4. Final answer generation

In [ ]:
# Example query for RouterR1
test_query = {"query": "What is the capital of France and what is its population?"}

print(f"Query: {test_query['query']}")
print("=" * 60)

# Route with details (includes token counts)
try:
    result = router.route_single(test_query, return_details=True)
    
    print("\nResponse:")
    print(result["response"])
    print("\nToken Usage:")
    print(f"  Prompt tokens: {result['prompt_tokens']}")
    print(f"  Completion tokens: {result['completion_tokens']}")
    print(f"  Route tokens: {result['route_tokens']}")
    print(f"  Total tokens: {result['total_tokens']}")
except Exception as e:
    print(f"Error during routing: {e}")

## 5. Batch Routing

In [ ]:
# Batch routing with multiple queries
batch_queries = [
    {"query": "What is machine learning?"},
    {"query": "Explain quantum computing in simple terms."},
    {"query": "How does photosynthesis work?"},
]

print(f"Processing {len(batch_queries)} queries...")
print("=" * 60)

try:
    results = router.route_batch(batch_queries)
    
    for i, result in enumerate(results, 1):
        print(f"\n{i}. Query: {result.get('query', 'N/A')[:50]}...")
        print(f"   Success: {result.get('success', False)}")
        print(f"   Response length: {len(result.get('response', ''))} chars")
        print(f"   Tokens: {result.get('prompt_tokens', 0) + result.get('completion_tokens', 0)}")
except Exception as e:
    print(f"Error during batch routing: {e}")

## 6. Task-Specific Routing

RouterR1 can format queries for specific tasks (MMLU, GSM8K, etc.).

In [ ]:
# Task-specific routing with ground truth for evaluation
task_query = {
    "query": "What is the derivative of x^2?",
    "task_name": "math",
    "ground_truth": "2x"
}

print(f"Task Query: {task_query['query']}")
print(f"Task: {task_query['task_name']}")
print(f"Ground Truth: {task_query['ground_truth']}")
print("=" * 60)

try:
    results = router.route_batch([task_query], task_name="math")
    result = results[0]
    
    print(f"\nResponse: {result.get('response', 'N/A')[:200]}...")
    if 'task_performance' in result:
        print(f"Task Performance: {result['task_performance']:.2f}")
except Exception as e:
    print(f"Error: {e}")

## 7. File-Based Inference

Load queries from a file and save results.

In [ ]:
import json

# Load queries from a JSONL file
def load_queries_from_file(file_path):
    """Load queries from a JSONL file."""
    queries = []
    with open(file_path, 'r', encoding='utf-8') as f:
        for line in f:
            if line.strip():
                queries.append(json.loads(line))
    return queries

# Save results to a JSONL file
def save_results_to_file(results, output_path):
    """Save routing results to a JSONL file."""
    os.makedirs(os.path.dirname(output_path), exist_ok=True)
    with open(output_path, 'w', encoding='utf-8') as f:
        for result in results:
            f.write(json.dumps(result, ensure_ascii=False) + '\n')
    print(f"Results saved to: {output_path}")

# Example: Load from default query file
QUERY_FILE = "data/example_data/query_data/default_query_test.jsonl"
OUTPUT_FILE = "outputs/router_r1_results.jsonl"

if os.path.exists(QUERY_FILE):
    # Load queries
    file_queries = load_queries_from_file(QUERY_FILE)
    print(f"Loaded {len(file_queries)} queries from: {QUERY_FILE}")
    
    # Route queries (limit to 5 for demo due to API costs)
    try:
        file_results = router.route_batch(file_queries[:5])
        print(f"Routed {len(file_results)} queries")
        
        # Save results
        save_results_to_file(file_results, OUTPUT_FILE)
        
        # Show sample results
        print(f"\nSample results:")
        for i, result in enumerate(file_results[:3], 1):
            print(f"  {i}. {result.get('query', '')[:40]}...")
            print(f"     Success: {result.get('success', False)}")
    except Exception as e:
        print(f"Error during batch routing: {e}")
else:
    print(f"Query file not found: {QUERY_FILE}")
    print("Create a JSONL file with format: {\"query\": \"Your question\"}")

## Summary

**RouterR1 Key Points**:
- Pre-trained agentic router (no training required)
- Uses vLLM for efficient GPU inference
- Iterative reasoning with external routing API
- Supports task-specific query formatting
- Token tracking for cost analysis

**Requirements**:
- CUDA-enabled GPU (vLLM requirement)
- Valid API credentials for routing service
- Sufficient GPU memory for the base model

**Use Cases**:
- Complex queries requiring reasoning
- Multi-step question answering
- Research and evaluation on routing strategies